In [1]:
import cv2
import mediapipe as mp
import threading
import numpy as np
import math
import tkinter as tk
from PIL import Image, ImageDraw, ImageFont
# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose

In [2]:

mp_drawing = mp.solutions.drawing_utils
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
pose_video2 = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

# Carregar vídeos
url = "http://192.168.1.42:8080/video"
# video_camera = cv2.VideoCapture(url, cv2.CAP_FFMPEG)
video_camera = cv2.VideoCapture('videoplayback.mp4')
video_file = cv2.VideoCapture('videoplayback.mp4')


barrier = threading.Barrier(2)

# Inicializar frames
frame_cam = None
frame_file = None
output_cam = None
output_file = None
landmarks_cam = None
landmarks_file = None
landmarks_cam_list = [] 
landmarks_file_list = []  




def calculate_similarity(landmarks1, landmarks2, max_distance=0.5):
    if not landmarks1 or not landmarks2:
        return None  

    distances = []
    for lm1, lm2 in zip(landmarks1, landmarks2):
        dist = math.sqrt((lm1.x - lm2.x)**2 + (lm1.y - lm2.y)**2 + (lm1.z - lm2.z)**2)
        distances.append(dist)

    mean_distance = np.mean(distances)

    similarity = max(0, 100 * (1 - mean_distance / max_distance)) 
    return similarity

# Função para processar a pose
def detectPose(image, pose):
    output_image = np.zeros_like(image)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        landmarks = results.pose_landmarks.landmark
    else:
        landmarks = None
    return output_image, landmarks

# Função para capturar e processar o frame da câmera
def process_camera():
    global frame_cam, output_cam, landmarks_cam
    barrier.wait()
    while video_camera.isOpened():
        ok, frame = video_camera.read()
        if not ok:
            break
        # frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (640, 480))
        frame_cam = frame
        output_cam, landmarks_cam = detectPose(frame, pose_video)
        if landmarks_cam:
            landmarks_cam_list.append(landmarks_cam_list.append(landmarks_cam))

def process_video():
    global frame_file, output_file, landmarks_file
    barrier.wait()
    while video_file.isOpened():
        ok2, frame2 = video_file.read()
        if not ok2:
            break
        frame2 = cv2.resize(frame2, (640, 480))
        frame_file = frame2
        output_file, landmarks_file = detectPose(frame2, pose_video2)
        if landmarks_file:
            landmarks_file_list.append(landmarks_file)

# Função para renderizar texto Unicode no frame usando Pillow
def render_text_with_pillow(frame, text, position, font_size, color):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)
    font = ImageFont.truetype("arial.ttf", font_size)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

# Iniciar threads
thread_cam = threading.Thread(target=process_camera)
thread_video = threading.Thread(target=process_video)

thread_cam.start()
thread_video.start()

# Loop para exibir as janelas
cv2.namedWindow('Video Original', cv2.WINDOW_NORMAL)
cv2.namedWindow('Meu Esqueleto', cv2.WINDOW_NORMAL)
cv2.namedWindow('Esqueleto do Video', cv2.WINDOW_NORMAL)

while True:
    # Mostrar os frames processados
    if frame_cam is not None and output_cam is not None:
        cv2.imshow('Meu Esqueleto', output_cam)
    if frame_file is not None and output_file is not None:
        cv2.imshow('Esqueleto do Video', output_file)       
        cv2.imshow('Video Original', frame_file)


    # Pressionar 'q' para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
video_camera.release()
video_file.release()
cv2.destroyAllWindows()

# Esperar as threads terminarem
thread_cam.join()
thread_video.join()
print(landmarks_cam_list)
print(landmarks_file_list)
similarities = []
for lm_cam, lm_file in zip(landmarks_cam_list, landmarks_file_list):
    if lm_cam and lm_file:
        similarity = calculate_similarity(lm_cam, lm_file)
        if similarity is not None:
            similarities.append(similarity)

if similarities:
    final_score = np.mean(similarities)
else:
    final_score = 0

# Criar uma imagem para exibir o resultado final
result_frame = np.zeros((400, 800, 3), dtype=np.uint8)  # Janela preta

# Adicionar textos ao frame
result_frame = render_text_with_pillow(result_frame, f"Sua nota final é: {final_score:.2f}%", (50, 150), 32, (255, 255, 255))
if final_score > 80:
    feedback = "Excelente desempenho!"
    color = (0, 255, 0)  # Verde
elif final_score > 50:
    feedback = "Bom desempenho. Continue praticando!"
    color = (0, 165, 255)  # Laranja
else:
    feedback = "Mais prática necessária."
    color = (0, 0, 255)  # Vermelho

result_frame = render_text_with_pillow(result_frame, feedback, (50, 250), 28, color)
cv2.imshow('Resultado Final', result_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


[[x: 0.521859825
y: 0.201659143
z: -0.32532233
visibility: 0.999988317
, x: 0.526910901
y: 0.187238067
z: -0.30214566
visibility: 0.999973178
, x: 0.530061
y: 0.186913624
z: -0.302210093
visibility: 0.999972582
, x: 0.534189463
y: 0.18667978
z: -0.302252918
visibility: 0.999970555
, x: 0.515265167
y: 0.187152714
z: -0.307689369
visibility: 0.999963284
, x: 0.510741174
y: 0.186859429
z: -0.307691485
visibility: 0.999961495
, x: 0.506036818
y: 0.186732113
z: -0.307682037
visibility: 0.999960303
, x: 0.538159311
y: 0.19006142
z: -0.14689596
visibility: 0.999943137
, x: 0.500732
y: 0.190125972
z: -0.172600344
visibility: 0.999933839
, x: 0.52874428
y: 0.216404766
z: -0.265151739
visibility: 0.999983788
, x: 0.514680326
y: 0.215506285
z: -0.272596091
visibility: 0.999981642
, x: 0.562706649
y: 0.283045411
z: -0.0283989869
visibility: 0.99992764
, x: 0.475749552
y: 0.276104242
z: -0.0749388635
visibility: 0.999859
, x: 0.565311372
y: 0.39843291
z: 0.0360757969
visibility: 0.894693732
, x: 0.